# そもそもどんなデータを扱うの？

本サイトで扱うデータについてまとめます．

## 環境構築

In [6]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [7]:
PATH_DATA = '../../data/preprocess/out/magazines.csv'

In [8]:
df = pd.read_csv(PATH_DATA)

## 概要

`df`は

- 週刊少年ジャンプ
- 週刊少年サンデー
- 週刊少年マガジン

の`1969-11-03`から`2017-07-12`までのすべてのマンガ作品のデータを格納した`DataFrame`です．サイズを見てみます．

In [24]:
df.shape

(135896, 12)

一行あたり一作品を格納するため，約14万行のデータとなっています．列は下記です．

In [10]:
df.columns

Index(['mcname', 'miname', 'cname', 'epname', 'creator', 'pageStart',
       'pageEnd', 'numberOfPages', 'datePublished', 'price', 'publisher',
       'editor'],
      dtype='object')

- `mcname`: 雑誌名
- `miname`: 雑誌巻号名
- `cname`: マンガ作品名
- `epname`: 各話タイトル
- `creator`: 作者名
- `pageStart`: 開始ページ
- `pageEnd`: 終了ページ
- `numberOfPages`: 雑誌の合計ページ数
- `datePublished`: 雑誌の発行日
- `price`: 雑誌の価格
- `publisher`: 雑誌の発行元
- `editor`: 雑誌の編集者（編集長）

冒頭数行を見てみましょう．

In [12]:
df.head(3).T

,0,1,2
mcname,週刊少年ジャンプ,週刊少年ジャンプ,週刊少年ジャンプ
miname,週刊少年ジャンプ 1969年 表示号数20,週刊少年ジャンプ 1969年 表示号数20,週刊少年ジャンプ 1969年 表示号数20
cname,ハレンチ学園,モサ,漫画コント55号
epname,赤い嵐の巻,盗まれた金の巻,ゆうかい魔の巻
creator,永井豪とダイナミックプロ,ちばてつや,榎本有也
pageStart,7.0,39.0,54.0
pageEnd,37.0,53.0,55.0
numberOfPages,296.0,296.0,296.0
datePublished,1969-11-03,1969-11-03,1969-11-03
price,90.0,90.0,90.0


`pandas`の`describe()`コマンドでざっくり集計してみます．

In [21]:
df.describe()

,pageStart,pageEnd,numberOfPages,price
count,135896.000000,135890.000000,135653.000000,135879.000000
mean,213.197213,230.853172,421.200740,202.327107
std,125.282380,123.402589,70.453981,42.839723
min,1.000000,0.000000,36.000000,70.000000
25%,109.000000,128.000000,356.000000,180.000000
50%,207.000000,224.000000,444.000000,210.000000
75%,307.000000,325.000000,474.000000,230.000000
max,2275.000000,1578.000000,600.000000,371.000000


- `pageStart`および`pageEnd`はすべての行でデータがありますが，それ以外は`NaN`（数値なし）が結構ありそうです
- `numberOfPages`の最小値，最大値が想定外に広がっていたので，あとで深堀りします．
- `price`も同様です．後で深堀りします．

次に，`NaN`の数を列ごとに集計します．

In [31]:
df.isna().sum()

mcname               0
miname               0
cname                8
epname           22000
creator            378
pageStart            0
pageEnd              6
numberOfPages      243
datePublished        0
price               17
publisher        17896
editor            1992
dtype: int64

- `cname`が不明なものが8行もあります．補完する手立てはありません．
- `epname`（各話タイトル）がない行が想定より多く，22000行に上ります．補完する手立てはありません．
- `creator`（作者）がない行が378行存在します．`cname`が同一のものがあれば補完できる可能性があります．
- `price`がない行が17行あります．前後の巻号から推定可能です．
- `publisher`がない行が17896行あります．`miname`が同一のものから補完可能です（そもそも`講談社`か`小学館`の2パターンしかありません）
- `editor`がない行が1992行あります．こちらも，前後の巻号から推定可能ですが，編集長が交代するタイミングだと打つ手がありません．